In [8]:
import os
import pandas as pd

new_packet_file = r"C:\Users\lenovo\NETGUARD-AI\data\real_time_packets.csv"

# Initialize CSV with headers if it doesn't exist
if not os.path.isfile(new_packet_file):
    pd.DataFrame(columns=['timestamp', 'src_ip', 'dst_ip', 'protocol', 'length']).to_csv(new_packet_file, index=False)


In [9]:
from scapy.all import sniff, IP
import datetime

packet_count = 0
max_packets = 1000

def capture_packet(packet):
    global packet_count
    if IP in packet and packet_count < max_packets:
        pkt_info = {
            'timestamp': datetime.datetime.now(),
            'src_ip': packet[IP].src,
            'dst_ip': packet[IP].dst,
            'protocol': packet[IP].proto,
            'length': len(packet)
        }
        # Append to CSV
        pd.DataFrame([pkt_info]).to_csv(new_packet_file, mode='a', header=False, index=False)
        packet_count += 1
        if packet_count >= max_packets:
            print(f"Captured {max_packets} packets. Stopping capture.")
            return True

# Start sniffing
sniff(iface='Wi-Fi', prn=capture_packet, store=False, stop_filter=lambda x: packet_count >= max_packets)


Captured 1000 packets. Stopping capture.
True


<Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>

In [17]:
import joblib

model = joblib.load(r"C:\Users\lenovo\NETGUARD-AI\models\netguard_rf_model.pkl")

try:
    print("Features used during training:")
    print(model.feature_names_in_)
except AttributeError:
    print("Feature names not stored in model. You might need to check your training script.")


Features used during training:
['index' 'Size_mean' 'Size_max' 'Size_min' 'Size_std' 'Timestamp_count']


In [19]:
import pandas as pd

# Load captured packets
packets = pd.read_csv(r"C:\Users\lenovo\NETGUARD-AI\data\real_time_packets.csv")

# Add index column
packets['index'] = range(len(packets))

# Aggregate Size statistics
size_mean = packets['length'].mean()
size_max = packets['length'].max()
size_min = packets['length'].min()
size_std = packets['length'].std()
timestamp_count = len(packets)

features_df = pd.DataFrame([{
    'index': packets['index'].iloc[-1],
    'Size_mean': size_mean,
    'Size_max': size_max,
    'Size_min': size_min,
    'Size_std': size_std,
    'Timestamp_count': timestamp_count
}])

print(features_df)

   index  Size_mean  Size_max  Size_min    Size_std  Timestamp_count
0    999     611.76      1514        43  650.314537             1000


In [22]:
output_file = r"C:\Users\lenovo\NETGUARD-AI\data\output.csv"

threat = model.predict(features_df)[0]

results_df = pd.DataFrame([{'threat': threat}])
results_df.to_csv(output_file, index=False)

print(f"Threat prediction saved to {output_file}")


Threat prediction saved to C:\Users\lenovo\NETGUARD-AI\data\output.csv


In [23]:
import pandas as pd
import joblib

# -----------------------------
# Step 1: Load captured packets
# -----------------------------
packets_file = r"C:\Users\lenovo\NETGUARD-AI\data\real_time_packets.csv"
packets = pd.read_csv(packets_file)

# Add index column
packets['index'] = range(len(packets))

# -----------------------------
# Step 2: Compute aggregated features
# -----------------------------
size_mean = packets['length'].mean()
size_max = packets['length'].max()
size_min = packets['length'].min()
size_std = packets['length'].std()
timestamp_count = len(packets)

# Prepare features DataFrame (single row for the entire captured set)
features_df = pd.DataFrame([{
    'index': packets['index'].iloc[-1],
    'Size_mean': size_mean,
    'Size_max': size_max,
    'Size_min': size_min,
    'Size_std': size_std,
    'Timestamp_count': timestamp_count
}])

print("Aggregated features for prediction:")
print(features_df)

# -----------------------------
# Step 3: Load pre-trained model
# -----------------------------
model_file = r"C:\Users\lenovo\NETGUARD-AI\models\netguard_rf_model.pkl"
model = joblib.load(model_file)

# Optional: Print features used during training
try:
    print("Features used during training:")
    print(model.feature_names_in_)
except AttributeError:
    print("Feature names not stored in model. Please check your training script.")

# -----------------------------
# Step 4: Predict threat
# -----------------------------
threat = model.predict(features_df)[0]

# Save prediction to CSV
output_file = r"C:\Users\lenovo\NETGUARD-AI\data\output.csv"
results_df = pd.DataFrame([{'threat': threat}])
results_df.to_csv(output_file, index=False)

print(f"Threat prediction saved to {output_file}")
print("Predicted threat:", threat)


Aggregated features for prediction:
   index  Size_mean  Size_max  Size_min    Size_std  Timestamp_count
0    999     611.76      1514        43  650.314537             1000
Features used during training:
['index' 'Size_mean' 'Size_max' 'Size_min' 'Size_std' 'Timestamp_count']
Threat prediction saved to C:\Users\lenovo\NETGUARD-AI\data\output.csv
Predicted threat: 0


In [28]:
import pandas as pd
import joblib

packets = pd.read_csv(r"C:\Users\lenovo\NETGUARD-AI\data\real_time_packets.csv")
model = joblib.load(r"C:\Users\lenovo\NETGUARD-AI\models\netguard_rf_model.pkl")

window_size = 10
results = []

for start in range(0, len(packets), window_size):
    window = packets.iloc[start:start+window_size]
    features = {
        'index': start,
        'Size_mean': window['length'].mean(),
        'Size_max': window['length'].max(),
        'Size_min': window['length'].min(),
        'Size_std': window['length'].std(),
        'Timestamp_count': len(window)
    }
    threat = model.predict(pd.DataFrame([features]))[0]
    features['threat'] = threat
    results.append(features)

results_df = pd.DataFrame(results)
results_df.to_csv(r"C:\Users\lenovo\NETGUARD-AI\data\threats_real_time.csv", index=False)
print("Threat predictions saved for each window!")


Threat predictions saved for each window!
